# Pipeline de Análise de Risco de Mercado

Este notebook contém o pipeline completo para a análise de risco de mercado de ativos brasileiros, seguindo os requisitos do desafio da Accenture.

A análise abrange a coleta de dados, o cálculo de indicadores de risco (volatilidade, VaR, correlação) e a visualização dos resultados de forma clara e interativa.

### 0. Importação das Bibliotecas

In [14]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1. Coleta de Dados Históricos

O primeiro passo do nosso pipeline é a aquisição dos dados. Coletamos as cotações diárias de fechamento dos últimos 6 meses para quatro ativos relevantes de diferentes setores do mercado brasileiro:

- **PETR4.SA:** Ação preferencial da **Petrobras**, representativa do setor de Commodities (Petróleo e Gás).
- **ITUB4.SA:** Ação preferencial do **Itaú Unibanco**, representativa do setor Financeiro.
- **MGLU3.SA:** Ação ordinária da **Magazine Luiza**, representativa do setor de Varejo e Consumo.
- **^BVSP:** O **Índice Bovespa**, que funciona como o principal benchmark do mercado de ações brasileiro.

In [15]:
# Definindo os tickers e o período
tickers = ['PETR4.SA', 'ITUB4.SA', 'MGLU3.SA', '^BVSP']
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(months=6)

# Baixando os dados de fechamento, que já são ajustados por padrão pela biblioteca yfinance
precos = yf.download(tickers, start=start_date, end=end_date)['Close']
precos = precos.dropna() # Garante que não há dias sem negociação para todos os ativos

print("Amostra dos preços dos ativos:")
precos.head()

/tmp/ipykernel_99315/348385888.py:7: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  4 of 4 completed

Amostra dos preços dos ativos:


Ticker,ITUB4.SA,MGLU3.SA,PETR4.SA,^BVSP
Date,,,,
2025-02-04,28.800358,7.028567,35.174004,125147.0
2025-02-05,29.237503,6.679081,34.918228,125534.0
2025-02-06,29.288931,7.174187,34.851913,126225.0
2025-02-07,29.074642,6.863533,34.652977,124619.0
2025-02-10,29.391790,7.047984,34.889809,125572.0


## 2. Cálculo dos Indicadores de Risco

Com os dados de preços em mãos, procedemos com o cálculo dos indicadores de risco solicitados.

### 2.1. Retornos Logarítmicos

Para analisar a variação dos ativos, calculamos os retornos diários. Utilizamos o retorno logarítmico por sua propriedade aditiva, que é matematicamente conveniente para análises de risco.

$$ R_t = \ln\left(\frac{P_t}{P_{t-1}}\right) $$

In [16]:
# Calculando os retornos diários
retornos = np.log(precos / precos.shift(1)).dropna()

print("Amostra dos retornos diários:")
retornos.head()

Amostra dos retornos diários:


Ticker,ITUB4.SA,MGLU3.SA,PETR4.SA,^BVSP
Date,,,,
2025-02-05,0.015064,-0.051002,-0.007298,0.003088
2025-02-06,0.001757,0.071509,-0.001901,0.005489
2025-02-07,-0.007343,-0.044267,-0.005724,-0.012805
2025-02-10,0.010849,0.026519,0.006811,0.007618
2025-02-11,0.010444,0.001376,0.000000,0.007537


### 2.2. Volatilidade, VaR e Correlação

- **Volatilidade Anualizada:** Mede a dispersão dos retornos. O desvio padrão dos retornos diários é anualizado para 252 dias úteis, a convenção para o mercado brasileiro.
$$ \sigma_{\text{anual}} = \text{desvio\_padrão}(R_{\text{diário}}) \times \sqrt{252} $$

- **Valor em Risco (VaR) Paramétrico a 95%:** Estima a perda máxima esperada em um único dia, com 95% de confiança, assumindo que os retornos seguem uma distribuição normal.
$$ \text{VaR}_{95\%} = -(\mu + Z_{0.05} \times \sigma) $$
Onde $\mu$ é a média dos retornos, $\sigma$ o desvio padrão e $Z_{0.05}$ o Z-score para o percentil 5%.

- **Correlação:** Mede a força e a direção da relação linear entre os retornos dos dois ativos.

In [17]:
# Volatilidade anualizada
volatilidade_anualizada = retornos.std() * np.sqrt(252)

# VaR Paramétrico (95% de confiança)
confianca = 0.95
media_retornos = retornos.mean()
desvio_padrao_retornos = retornos.std()
z_score = norm.ppf(1 - confianca)
var_parametrico = -(media_retornos + z_score * desvio_padrao_retornos)

# Correlação entre os ativos
correlacao = retornos.corr()

## 3. Apresentação e Visualização dos Resultados

Nesta seção, apresentamos os resultados de forma visual e tabular para uma interpretação clara e direta da análise de risco.

In [18]:
# Definindo um mapa de cores padronizado para consistência visual em todos os gráficos
color_map = {
    'PETR4.SA': 'green',   # Petrobras em verde
    'ITUB4.SA': 'orange',  # Itaú em laranja
    'MGLU3.SA': 'blue',  # Magazine Luiza em azul
    '^BVSP': 'red'      # Ibovespa em vermelho
}

### 3.1. Resumo dos Indicadores

A tabela abaixo serve como um resumo executivo da nossa análise, consolidando os principais indicadores de risco para cada ativo.

In [19]:
tabela_indicadores = pd.DataFrame(
    {
        'Volatilidade Anualizada': volatilidade_anualizada,
        'VaR Diário (95%)': var_parametrico
    }
).T # Transpõe o DataFrame para o formato desejado

# Formatando os valores como percentagens
tabela_formatada = tabela_indicadores.applymap('{:.2%}'.format)

# Criando a figura com Plotly
fig_tabela = go.Figure(data=[go.Table(
    header=dict(values=['Indicador'] + list(tabela_formatada.columns),
                fill_color='navy',
                align='left', font=dict(color='white', size=14)),
    cells=dict(values=[tabela_formatada.index] + [tabela_formatada[col] for col in tabela_formatada.columns],
               fill_color='lightgrey',
               align='left', font=dict(color='black', size=12)))
])

fig_tabela.update_layout(title_text='Tabela de Indicadores de Risco', title_x=0.5)
fig_tabela.show()

/tmp/ipykernel_99315/2779988750.py:9: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



### 3.2. Análise de Desempenho

Analisamos a evolução dos preços dos ativos ao longo do período. Como os ativos possuem escalas de valor muito diferentes (Reais vs. Pontos), a melhor prática é visualizá-los num gráfico com dois eixos Y. O segundo gráfico, normalizado para uma base 100, permite uma comparação justa do desempenho percentual.

In [20]:
# Criando a figura com um eixo Y secundário
fig_precos = make_subplots(specs=[[{"secondary_y": True}]])

# Adicionando as ações (eixo primário, à esquerda)
for ticker in ['PETR4.SA', 'ITUB4.SA', 'MGLU3.SA']:
    fig_precos.add_trace(
        go.Scatter(x=precos.index, y=precos[ticker], name=f'{ticker} (R$)', line=dict(color=color_map.get(ticker))),
        secondary_y=False,
    )

# Adicionando o Ibovespa (eixo secundário, à direita)
fig_precos.add_trace(
    go.Scatter(x=precos.index, y=precos['^BVSP'], name='^BVSP (Pontos)', line=dict(color=color_map.get('^BVSP'))),
    secondary_y=True,
)

# Adicionando títulos e formatando os eixos
fig_precos.update_layout(
    title_text='Série Histórica de Preços (Últimos 6 Meses)'
)
fig_precos.update_xaxes(title_text='Data')
fig_precos.update_yaxes(title_text='Preço Ações (R$)', secondary_y=False)
fig_precos.update_yaxes(title_text='Pontuação ^BVSP', secondary_y=True)

fig_precos.show()

In [21]:
precos_normalizados = (precos / precos.iloc[0]) * 100
fig_precos_normalizados = px.line(precos_normalizados, title='Desempenho Comparativo (Base 100)',
                                    labels={'value': 'Valor (Base 100)', 'variable': 'Ativo'},
                                    color_discrete_map=color_map)
fig_precos_normalizados.show()

### 3.3. Análise de Risco

Os gráficos a seguir aprofundam a análise de risco. A volatilidade móvel mostra como o "nervosismo" do mercado mudou ao longo do tempo, enquanto o drawdown quantifica a maior perda percentual que um investidor teria sofrido a partir de um pico anterior.

In [22]:
vol_movel = retornos.rolling(window=21).std() * np.sqrt(252)
fig_vol_movel = px.line(vol_movel, title='Volatilidade Móvel Anualizada (21 dias)',
                        labels={'value': 'Volatilidade', 'variable': 'Ativo'},
                        color_discrete_map=color_map)
fig_vol_movel.show()

In [ ]:
retorno_acumulado = (1 + retornos).cumprod()
pico_anterior = retorno_acumulado.cummax()
drawdown = (retorno_acumulado - pico_anterior) / pico_anterior

# Criamos a figura de forma mais controlada com graph_objects
fig_drawdown = go.Figure()

# Adicionamos uma linha para cada ativo
for ticker in drawdown.columns:
    fig_drawdown.add_trace(go.Scatter(
        x=drawdown.index, 
        y=drawdown[ticker], 
        fill='tozeroy',
        mode='lines', 
        name=ticker,
        line=dict(color=color_map.get(ticker))
    ))

# Atualizamos o layout para ficar mais claro
fig_drawdown.update_layout(
    title='Drawdown dos Ativos',
    yaxis_title='Queda Percentual desde o Pico',
    xaxis_title='Data',
    legend_title='Ticker'
)

fig_drawdown.show()

### 3.4. Análise Estatística

Finalmente, os gráficos estatísticos. O heatmap visualiza a correlação, enquanto o histograma nos ajuda a entender a distribuição dos retornos diários, o que é fundamental para a validade do nosso cálculo de VaR paramétrico.

In [24]:
fig_retornos = px.line(retornos, title='Retornos Diários dos Ativos',
                       labels={'value': 'Retorno Logarítmico', 'variable': 'Ativo'},
                       color_discrete_map=color_map)
fig_retornos.show()

In [25]:
fig_heatmap = px.imshow(correlacao, text_auto=True, title='Heatmap de Correlação entre os Ativos')
fig_heatmap.show()

In [26]:
# Para aplicar o mapa de cores, precisamos 'derreter' o dataframe primeiro
retornos_melted = retornos.melt(var_name='Ticker', value_name='Retorno Diário')

fig_hist = px.histogram(retornos_melted, x='Retorno Diário', color='Ticker',
                        marginal='box', 
                        color_discrete_map=color_map,
                        title='Distribuição dos Retornos Diários')
fig_hist.show()

## 4. Conclusão da Análise

A análise multissetorial confirma que os ativos possuem perfis de risco distintos. A Magazine Luiza (MGLU3.SA) destaca-se como a mais volátil, com o maior VaR e o maior drawdown, refletindo o risco elevado do setor de varejo. A Petrobras (PETR4.SA) também apresenta risco acima da média do mercado, influenciada por fatores de commodities e geopolíticos. O Itaú Unibanco (ITUB4.SA), por outro lado, mostra-se como o ativo de menor risco entre as empresas analisadas, com volatilidade e VaR mais próximos aos do Ibovespa. A correlação positiva entre todas as ações e o índice indica que todos são influenciados pelo risco sistêmico do mercado brasileiro, mas em graus variados.